<a href="https://colab.research.google.com/github/bierooed/phrase-search-algorithm/blob/master/Oganisyan_Oganes_Copy_of_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download and unzip the English texts collection, which contains over 100,000 documents. The documents will be saved in the /content/enwiki directory

In [ ]:
!gdown https://drive.google.com/uc?id=1wzj7AXu_UmwrMc5Tl1_acMKbYdKWaXMC
!unzip /content/enwiki.zip

# Section A

Simple code for searching phrases

In [ ]:
# This code block prepares documents for further searching

from typing import List, Dict
from tqdm import tqdm
import os

def read_text_fragments(file_path: str) -> List[str]:
    with open(file_path, "r") as f:
        fragments = f.readlines()
    # Remove leading and trailing spaces and new line characters before returning text fragments
    return [fragment.strip() for fragment in fragments if fragment.strip()]

def extract_and_combine_text_fragments(folder_path: str) -> Dict[str, List[str]]:
    """
    This function iterates over all files in the folder and saves their fragments in a dictionary,
    Returns:
        A dictionary, where the dictionary key is the file name, and its value is a list of all fragments in that file.
    Example:
        texts_fragments_dict = {
              "file_1.txt": ["this is an example of some fragment", "this is an example of another fragment"],
              "file_2.txt": ["this is another example"],
              ...
        }
    """

    texts_fragments_dict = {}
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Extract text file fragments
        file_fragments = read_text_fragments(file_path)
        # Add file name and its value to texts_fragments_dict
        texts_fragments_dict[file_name] = file_fragments
    return texts_fragments_dict

# Extract all documents fragments from the specified folder
folder_path = "/content/enwiki"
texts_fragments_dict = extract_and_combine_text_fragments(folder_path=folder_path)

In [ ]:
# Function that searches similar phrases
def search_similar_phrases(phrase: str, texts_fragments: Dict[str, List[str]]) -> List[List[str]]:
    """
    For the given phrase, search for similar phrases in the texts_fragments dict.
    Returns:
        A list containing information about in which file the phrase is written and the corresponding fragment.
        If nothing is found, it returns an empty list.
    Example:
        search_results = [
            ["file_1.txt", "this is an example of some fragment"],
            ["file_1.txt", "this is an example of another fragment"],
            ["file_2.txt", "this is another example"]
            ...
        ]
    """
    search_results = []
    # Iterate over all files
    for file_name in texts_fragments:
        # Iterate over file fragments
        for fragment in texts_fragments[file_name]:
            # Check if the phrase is in our fragment, then save it in the search_results list
            if phrase in fragment:
                search_results.append([file_name, fragment])

    return search_results

In [ ]:
# Example of searching
phrase = "Berisha is elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[['17919119.txt', 'Berisha is elected chairman of the Democratic Party.']]


This algorithm works slowly; if we have, for example, a million texts, it can take minutes to return an answer.

Besides this, the proposed method only finds identical texts.

In [ ]:
# 1. If we delete some of the middle words, we will receive an empty result
phrase = "Berisha elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 2. Or if we delete some word ending
phrase = "Berisha is elect chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 3. Or if we make words lowercase
phrase = "berisha is elected chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


In [ ]:
# 4. Or if we change some similar meaning words, "elected" -> "appointed"
phrase = "Berisha is appointed chairman"
search_results = search_similar_phrases(phrase, texts_fragments_dict)
print(search_results)

[]


# Section B

Research and implement a search algorithm that overcomes these limitations and performs efficiently and rapidly on a vast collection of documents. Additionally, you need to explore other potential problems that might occur during the search process and propose solutions to avoid them.


In [ ]:
from typing import List, Dict
import os

def read_text_fragments(file_path: str) -> List[str]:
    with open(file_path, 'r') as f:
        fragments = f.readlines()
        return [fragment.strip() for fragment in fragments if fragment.strip()]


def extract_and_combine_text_fragments(folder_path: str) -> Dict[str, List[str]]:
    texts_fragments_dict = {}

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            fragments = read_text_fragments(file_path)
            texts_fragments_dict[file_name] = fragments
    return texts_fragments_dict


class InvertedIndex:
    def __init__(self):
        self.index = {}

    def add_document(self, doc_id, fragment):
        translation_table = str.maketrans("", "", ".,()")  # Таблица для удаления символов
        cleaned_fragment = fragment.translate(translation_table)
        tokens = cleaned_fragment.lower().split()
        for token in tokens:
            if token not in self.index:
                self.index[token] = []
            self.index[token].append(doc_id)

    def search(self, query):
        query_tokens = query.lower().split()
        doc_matches = {}
        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    if doc_id not in doc_matches:
                        doc_matches[doc_id] = set()
                    doc_matches[doc_id].add(token)

        result = []
        for doc_id, matched_tokens in doc_matches.items():
            if len(matched_tokens) == len(query_tokens):
                result.append(doc_id)

        return result

    def get_fragment(self, doc_id, fragment_idx):
        if doc_id in documents and 0 <= fragment_idx < len(documents[doc_id]):
            return documents[doc_id][fragment_idx]
        return None


documents = extract_and_combine_text_fragments('./text_files')

inverted_index = InvertedIndex()

for doc_id, fragments in documents.items():
    for fragment in fragments:
        inverted_index.add_document(doc_id, fragment)

query = "Berisha is elected chairman of the Democratic Party"
print(len(query.split()))
search_results = inverted_index.search(query)
print(search_results, ' ---')
for doc_id in search_results:
    matching_fragments = [fragment for fragment in documents[doc_id] if sum(1 for token in query.lower().split() if token in fragment.lower()) >= len(query.split())]
    for fragment in matching_fragments:
        print(f"Found in document: {doc_id}, fragment: {fragment}")


